# Time Data Types

Contents

    Time and Date Variables
    Time Operations

The time data types are broken out into a separate section from the introductory section on data types. (Basic Data Types) The reason for this is that dealing with time data can be subtle and must be done carefully because the data type can be cast in a variety of different ways. It is not an introductory topic, and if not done well can scare off the normal people.

I will first go over the basic time data types and then explore the different kinds of operations that are done with the time data types. Please be cautious with time data and read the complete description including the caveats. There are some common mistakes that result in calculations that yield results that are very different from the intended values.

## Time and Date Variables

There are a variety of different types specific to time data fields in R. Here we only look at two, the POSIXct and POSIXlt data types:

POSIXct

    The POSIXct data type is the number of seconds since the start of January 1, 1970. Negative numbers represent the number of seconds before this time, and positive numbers represent the number of seconds afterwards.

POSIXlt

    The POSIXlt data type is a vector, and the entries in the vector have the following meanings:

        seconds
        minutes
        hours
        day of month (1-31)
        month of the year (0-11)
        years since 1900
        day of the week (0-6 where 0 represents Sunday)
        day of the year (0-365)
        Daylight savings indicator (positive if it is daylight savings)

Part of the difficulty with time data types is that R prints them out in a way that is different from how it stores them internally. This can make type conversions tricky, and you have to be careful and test your operations to insure that R is doing what you think it is doing.

To get the current time, the Sys.time() can be used, and you can play around a bit with the basic types to get a feel for what R is doing. The as.POSIXct and as.POSIXlt commands are used to convert the time value into the different formats.

In [1]:
help(DateTimeClasses)

In [2]:
t <- Sys.time()
typeof(t)
t

[1] "double"

[1] "2019-03-09 12:53:20 BDT"

In [3]:
print(t)

[1] "2019-03-09 12:53:20 BDT"


In [4]:
cat(t,"\n")

1552114400 


In [5]:
c <- as.POSIXct(t)
typeof(c)
print(c)

[1] "double"

[1] "2019-03-09 12:53:20 BDT"


In [6]:
cat(c,"\n")

1552114400 


In [7]:
l <- as.POSIXlt(t)
l
typeof(l)
print(l)

[1] "2019-03-09 12:53:20 BDT"

[1] "list"

[1] "2019-03-09 12:53:20 BDT"


In [8]:
cat(l,"\n")

ERROR: Error in cat(l, "\n"): argument 1 (type 'list') cannot be handled by 'cat'


In [9]:
names(l)

NULL

In [10]:
l[[1]]

[1] 20.01803

In [11]:
l[[2]]

[1] 53

In [12]:
l[[3]]

[1] 12

In [13]:
l[[4]]

[1] 9

In [14]:
l[[5]]

[1] 2

In [15]:
l[[6]]

[1] 119

In [16]:
l[[7]]

[1] 6

In [17]:
l[[8]]

[1] 67

In [18]:
l[[9]]

[1] 0

In [19]:
b <- as.POSIXct(l)
cat(b,"\n")

1552114400 


There are times when you have a time data type and want to convert it into a string so it can be saved into a file to be read by another application. The strftime command is used to take a time data type and convert it to a string. You must supply an additional format string to let R what format you want to use. See the help page on strftime to get detailed information about the format string.

In [20]:
help(strftime)

In [21]:
t <- Sys.time()
cat(t,"\n")

1552114762 


In [23]:
timeStamp <-  strftime(t,"%Y-%m-%d %H:%M:%S")
timeStamp
typeof(timeStamp)

[1] "2019-03-09 12:59:21"

[1] "character"

Commonly a time stamp is saved in a data file, and it must be converted into a time data type to allow for calculations. For example, you may be interested in how much time has elapsed between two observations. The strptime command is used to take a string and convert it into a time data type. Like strftime it requires a format string in addition to the time stamp.

The strptime command is used to take a string and convert it into a form that R can use for calculations. In the following example a data frame is defined that has the dates stored as strings. If you read the data in from a csv file this is how R will keep track of the data. Note that in this context the strings are assumed to represent ordinal data, and R will assume that the data field is a set of factors. You have to use the strptime command to convert it into a time field.

In [24]:
myData <- data.frame(time=c("2014-01-23 14:28:21","2014-01-23 14:28:55",
                              "2014-01-23 14:29:02","2014-01-23 14:31:18"),
                      speed=c(2.0,2.2,3.4,5.5))
myData

time,speed
2014-01-23 14:28:21,2.0
2014-01-23 14:28:55,2.2
2014-01-23 14:29:02,3.4
2014-01-23 14:31:18,5.5


In [25]:
summary(myData)

                  time       speed      
 2014-01-23 14:28:21:1   Min.   :2.000  
 2014-01-23 14:28:55:1   1st Qu.:2.150  
 2014-01-23 14:29:02:1   Median :2.800  
 2014-01-23 14:31:18:1   Mean   :3.275  
                         3rd Qu.:3.925  
                         Max.   :5.500  

In [26]:
myData$time[1]

[1] 2014-01-23 14:28:21
4 Levels: 2014-01-23 14:28:21 2014-01-23 14:28:55 ... 2014-01-23 14:31:18

In [27]:
typeof(myData$time[1])

[1] "integer"

In [28]:
myData$time <- strptime(myData$time,"%Y-%m-%d %H:%M:%S")
myData

time,speed
2014-01-23 14:28:21,2.0
2014-01-23 14:28:55,2.2
2014-01-23 14:29:02,3.4
2014-01-23 14:31:18,5.5


In [29]:
myData$time[1]

[1] "2014-01-23 14:28:21 BDT"

In [30]:
typeof(myData$time[1])

[1] "list"

In [31]:
myData$time[1][[2]]

[1] 28

Now you can perform operations on the fields. For example you can determine the time between observations. (Please see the notes below on time operations. This example is a bit misleading!)

In [32]:
N = length(myData$time)
myData$time[2:N] - myData$time[1:(N-1)]

Time differences in secs
[1]  34   7 136

In addition to the time data types R also has a date data type. The difference is that the date data type keeps track of numbers of days rather than seconds. You can cast a string into a date type using the as.Date function. The as.Date function takes the same arguments as the time data types discussed above.

In [50]:
lct <- Sys.getlocale("LC_TIME"); Sys.setlocale("LC_TIME", "C")
thedates <- c("1 jan 2012","1 jan 2013","1 jan 2014")
dateFields <- as.Date(thedates,"%d %b %Y")
typeof(dateFields)
dateFields
N <- length(dateFields)
N
diff <- dateFields[1:(N-1)] - dateFields[2:N]
diff

[1] "C"

[1] "double"

[1] "2012-01-01" "2013-01-01" "2014-01-01"

[1] 3

Time differences in days
[1] -366 -365

You can also define a date in terms of the number days after another date using the origin option.

In [58]:
infamy <- as.Date(-179,origin="1985-06-29")
infamy

[1] "1985-01-01"

In [59]:
today <- Sys.Date()
today

[1] "2019-03-09"

In [60]:
today-infamy

Time difference of 12485 days

Finally, a nice function to know about and use is the format command. It can be used in a wide variety of situations, and not just for dates. It is helpful for dates, though, because you can use it in cat and print statements to make sure that your output is in exactly the form that you want.

In [61]:
theTime <- Sys.time()
theTime


[1] "2019-03-09 13:21:47 BDT"

In [62]:
a <- rexp(1,0.1)
a

[1] 11.49618

In [63]:
cat("At about",format(theTime,"%H:%M"),"the time between occurances was around",format(a,digits=3),"seconds\n")

At about 13:21 the time between occurances was around 11.5 seconds


## Time Operations

The most difficult part of dealing with time data can be converting it into the right format. Once a time or date is stored in R’s internal format then a number of basic operations are available. The thing to keep in mind, though, is that the units you get after an operation can vary depending on the magnitude of the time values. Be very careful when dealing with time operations and vigorously test your codes.

In [64]:
now <- Sys.time()
now

now-60

earlier <- strptime("2000-01-01 00:00:00","%Y-%m-%d %H:%M:%S")
later <- strptime("2000-01-01 00:00:20","%Y-%m-%d %H:%M:%S")
later-earlier

as.double(later-earlier)

earlier <- strptime("2000-01-01 00:00:00","%Y-%m-%d %H:%M:%S")
later <- strptime("2000-01-01 01:00:00","%Y-%m-%d %H:%M:%S")
later-earlier

as.double(later-earlier)

up <- as.Date("1961-08-13")
down <- as.Date("1989-11-9")
down-up

[1] "2019-03-09 13:24:41 BDT"

[1] "2019-03-09 13:23:41 BDT"

Time difference of 20 secs

[1] 20

Time difference of 1 hours

[1] 1

Time difference of 10315 days

The two examples involving the variables earlier and later in the previous code sample should cause you a little concern. The value of the difference depends on the largest units with respect to the difference! The issue is that when you subtract dates R uses the equivalent of the difftime command. We need to know how this operates to reduce the ambiguity when comparing times.

In [65]:
earlier <- strptime("2000-01-01 00:00:00","%Y-%m-%d %H:%M:%S")
later <- strptime("2000-01-01 01:00:00","%Y-%m-%d %H:%M:%S")
difftime(later,earlier)

difftime(later,earlier,units="secs")

Time difference of 1 hours

Time difference of 3600 secs

One thing to be careful about difftime is that it is a double precision number, but it has units attached to it. This can be tricky, and you should be careful about the ambiguity in using this command. I personally always try to specify the units to avoid this.

In [66]:
earlier <- strptime("2000-01-01 00:00:00","%Y-%m-%d %H:%M:%S")
later <- strptime("2000-01-01 00:00:20","%Y-%m-%d %H:%M:%S")
d <- difftime(later,earlier)
d

typeof(d)

as.double(d)

Time difference of 20 secs

[1] "double"

[1] 20

Another way to define a time difference is to use the as.difftime command. It takes two dates and will compute the difference between them. It takes a time, its format, and the units to use. Note that in the following example R is able to figure out what the units are when making the calculation.

In [67]:
diff <- as.difftime("00:30:00","%H:%M:%S",units="hour")
diff

Sys.time()

Sys.time()+diff

Time difference of 0.5 hours

[1] "2019-03-09 13:27:53 BDT"

[1] "2019-03-09 13:57:53 BDT"

The last thing to mention is that once a time stamp is cast into one of R’s internal formats comparisons can be made in a natural way.

In [68]:
diff <- as.difftime("00:30:00","%H:%M:%S",units="hour")
now <- Sys.time()
later <- now + diff
now

later


if(now < later)
    {
        cat("there you go\n")
    }

[1] "2019-03-09 13:29:29 BDT"

[1] "2019-03-09 13:59:29 BDT"

there you go
